# Resistance/Support AND Candles Patterns

### Import libraries

In [1]:
import pandas as pd

### Load OHLCV data

In [2]:
tickers_list = ['AAPL']

df_tics = pd.read_hdf("datasets/df_SnP_500_ohlcv.h5", "df", mode = 'r')
df_tics = df_tics[df_tics['tic'].isin(tickers_list)]

In [3]:
df = df_tics.drop(columns=['tic','adj_close'],axis=1)
df

,date,open,high,low,close,volume
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,102223600
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,118071600
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,89738400
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,94640000
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,82271200
...,...,...,...,...,...,...
1420,2023-08-24,180.669998,181.100006,176.009995,176.380005,54945800
1421,2023-08-25,177.380005,179.149994,175.820007,178.610001,51418700
1422,2023-08-28,180.089996,180.589996,178.550003,180.190002,43820700
1423,2023-08-29,179.699997,184.899994,179.500000,184.119995,53003900


### Handling missing values

In [4]:
#Check if NA values are in data
df = df[df['volume']!=0]
df = df.reset_index(drop=True)
print(df.isna().sum())
print(df.tail())

date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64
           date        open        high         low       close    volume
1420 2023-08-24  180.669998  181.100006  176.009995  176.380005  54945800
1421 2023-08-25  177.380005  179.149994  175.820007  178.610001  51418700
1422 2023-08-28  180.089996  180.589996  178.550003  180.190002  43820700
1423 2023-08-29  179.699997  184.899994  179.500000  184.119995  53003900
1424 2023-08-30  184.940002  187.850006  184.740005  187.649994  60754700


# Support and Resistance FUNCTIONS

In [5]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1

In [6]:
length = len(df)
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [7]:
n1=2
n2=2
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
            
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



In [8]:
df['signal']=signal
df.tail()

,date,open,high,low,close,volume,signal
1420,2023-08-24,180.669998,181.100006,176.009995,176.380005,54945800,0
1421,2023-08-25,177.380005,179.149994,175.820007,178.610001,51418700,0
1422,2023-08-28,180.089996,180.589996,178.550003,180.190002,43820700,0
1423,2023-08-29,179.699997,184.899994,179.500000,184.119995,53003900,0
1424,2023-08-30,184.940002,187.850006,184.740005,187.649994,60754700,0


### Count number of BUY and SELL signals

- 1: SELL 
- 2: BUY

In [9]:
print(f"Number of SELL signals = {len(df[df['signal']==1])}")
print(f"Number of BUY signals = {len(df[df['signal']==2])}")

Number of SELL signals = 24
Number of BUY signals = 16


### Modify column names to fit with Backtesting package

In [10]:
df.columns = ['Local time', 'Open', 'High', 'Low', 'Close', 'Volume', 'signal']
df = df.iloc[10:]
df

,Local time,Open,High,Low,Close,Volume,signal
300,2019-03-14,45.974998,46.025002,45.639999,45.932499,94318000,0
301,2019-03-15,46.212502,46.832500,45.935001,46.529999,156171600,0
302,2019-03-18,46.450001,47.097500,46.447498,47.005001,104879200,0
303,2019-03-19,47.087502,47.247501,46.480000,46.632500,126585600,0
304,2019-03-20,46.557499,47.372501,46.182499,47.040001,124140800,0
...,...,...,...,...,...,...,...
1420,2023-08-24,180.669998,181.100006,176.009995,176.380005,54945800,0
1421,2023-08-25,177.380005,179.149994,175.820007,178.610001,51418700,0
1422,2023-08-28,180.089996,180.589996,178.550003,180.190002,43820700,0
1423,2023-08-29,179.699997,184.899994,179.500000,184.119995,53003900,0


In [11]:
def SIGNAL():
    return df.signal

In [12]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [13]:
from backtesting import Strategy

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - 750e-4
            tp1 = self.data.Close[-1] + 600e-4
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + 750e-4
            tp1 = self.data.Close[-1] - 600e-4
            self.sell(sl=sl1, tp=tp1)

In [14]:
from backtesting import Backtest

bt = Backtest(df, MyCandlesStrat, cash=10000, commission=.002)
stat = bt.run()
stat



C:\Users\erkun\AppData\Local\Temp\ipykernel_21768\776730442.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyCandlesStrat, cash=10000, commission=.002)


ValueError: Short orders require: TP (51.73000091552734) < LIMIT (51.68642091369629) < SL (51.86500091552735)

In [15]:
bt.plot()

RuntimeError: First issue `backtest.run()` to obtain results.